In [1]:
import numpy as np
import pandas as pd
import os.path
import subprocess
import math
import datetime
import matplotlib
import os.path
from copy import copy, deepcopy
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime, timedelta

In [2]:
#settings
start_date=datetime(2017,12,6)
num_days=34
start_date_scen=start_date+timedelta(days=num_days)
num_days_scen=14

In [3]:

dtdz=-0.006       # temperature gradient (deg C km-1)
hstat=1709         # elevation of weather station
topo=[0.78,1768,44,0.0172]        # area, elevation, max depth, volume
elev_lake=topo[1]     # elevation of lake

# ---- scenario - INPUT / ADJUSTMENT / CALIBRATION
scenprec_corrfact=1.0              # Factor to correct precipitation in second week of scenario - normally strongly overestimated...

ice_thickness_scen=25              # (cm) Ice thickness (measured) at start of scenario period  (default: -9999)
#ice_thickness_scen=-np.inf

snow_thickness_scen=3             # (cm w.==!) snow thickness (measured) at start of scenario period  (default: -9999)
#snow_thickness_scen=-np.inf

snow_dens_scen=400                # (kg m-3) snow density (measured/estimated)  at start of scenario period  (default: -9999)
#snow_dens_scen=-np.inf

slushice_thickness_scen=-np.inf               # (cm) thickness of refrozen ice ("Sandwich") at start of scenario period  (default: -9999)
#slushice_thickness_scen=-np.inf

# ------------------


format_csv='y'      # raw climate file in csv format

# prepare climate data file
prepare='n'

twater_ini=[4,4,20]       # [Tmin,Tmax,break(m)]

int_slush=[datetime(2018,1,3),datetime(2018,1,4),datetime(2018,1,9)]       # dates to insert slush

vol_slush=[0.003,0.017,0.01,-np.inf]            # volumes of inserted slush in m w.e. (same length as int_slush)

dtdz=-0.006       # temperature gradient (deg C km-1)
hstat=1709         # elevation of weather station

alpha_ice=0.3         # albedo of ice
alpha_water=0.12      # albedo of water
alpha_snow0=0.92      # albedo of fresh snow
alpha_snow1=0.5       # albedo of very old snow

turbulent_fact=3.5  # 5 # factor to increase transfer of water heat to base of ice (to be calibrated)

corrprec=1      # Korrektur des Niederschlags

lakedepth=25     # depth of lake m

first_freezeparam=1.0        # [-] Parameter to accelerate or decelerate the first freezing of the lake

# temperature of river entering the lake and lake geometry
topo=[0.78,1768,44,0.0172]        # area, elevation, max depth, volume
flow=1     #5.8                    # [m3/s] river charactersitic inflow
flow_tcorr=-5    # [deg C] correct temperature of stream flow for elevation / snow melt relative to air temperature
flow_tmin=0      # [deg C] minimum temperature of stream

wwin=[1.2,1,0.8]     # weighting of lake water layering in different depths

elev_lake=topo[1]     # elevation of lake


# number of iterations  for heat conduction (per hour)
iterations_si=600     # snow / ice
iterations_w=50     # water

# number of iterations for snow surface temperature
dt_snoit=120    # s

freq_plots=6     # (hours) frequency of output plots
language_plots='e'      # language of plots (d: deutsch, e: english)


# -----------------------------
# parameters / constants

kice=2.33       # conductivity ice [J s-1 K-1 m-1]
kwater=0.56     # conductivity water
kair=0.001

cice=1890000     # heat capacity of ice
cair=1297
cwater=4217700    # [J m-3 K-1]

emissivity_atm=0.95
emissivity_ice=0.966
emissivity_water=0.957
emissivity_snow=0.97

attenuation_water=1       # m-1
attenuation_ice0=1.0         # m-1
attenuation_snow=10     # m-1   ??? 10
at_rho=250
fact_at=0.1                # increase in ice attenuation per day (ageing)

rho=917   # ice density
rho_water=1000
rho_snow0=70    # density of fresh snow
rho_max=400

compression='y'
rho_compression=450    # kg m-3


L=334000*rho    # [J m-3] latent heat of fusion
Lhe=2260000    # [J kg-1] latent heat of evaporation
Lhs=2594000    # [J kg-1] latent heat of sublimation

bulk_exchange_s=0.3*pow(10,-3)      # bulk exchange coefficients for sensible heat flux (to be calibrated)
bulk_exchange_l=0.8*pow(10,-3)      # bulk exchange coefficients for latent heat flux (to be calibrated)

bulk_exchange_s=1.5*pow(10,-3)      # bulk exchange coefficients for sensible heat flux (to be calibrated)
bulk_exchange_l=1.5*pow(10,-3)      # bulk exchange coefficients for latent heat flux (to be calibrated)


# -----------------------
# settings

dz=0.005      # nodespacing (ice)
dzs=0.0025    # snow
dzw=0.2      # water

nodes=[lakedepth/dzw,300,300]     # water, ice, snow

dt=3600     # [s] timestep

s_b=5.67*pow(10,-8)    # stefan boltzmann

t_wmin=4     # minimum temperature of water (density)

meltlast=0

In [4]:
# read files
df=pd.read_csv('../data/raw/samedan_2018.dat',header=None,encoding='latin-1',skiprows=14, sep='\s+',names=['STA','Year','Mo','Day','HH','MM','Tair','RH','Wind','Rad','Pressure','Prec'])
df=df.replace(32767, np.NaN)    #Assign missing data as null
df=df.drop('STA',axis=1)
error=df.isnull()  # Grab DataFrame rows where column has errors
right=error[error['Tair']].index.tolist()
a=np.array(right)
b=a-1
b[0]=0
columns=['Tair','RH','Wind','Rad','Pressure','Prec']

df['Year']=df['Year'].astype(int)
df['Mo']=df['Mo'].astype(int)
df['Day']=df['Day'].astype(int)
for i in range(0, a.size):
    df.loc[a[i],columns]=df.loc[b[i],columns] #Use previous data to fill #Error last column not filled

df.loc[0,columns]=df.loc[1,columns]
for i in range(0, df.shape[0]):
    t = datetime(df['Year'][i], df['Mo'][i], df['Day'][i],df['HH'][i], 0)
    df.loc[i,'When']=t
    df.loc[i,'DoY']=t.timetuple().tm_yday
df['DoY']=df['DoY'].astype(int)
df['When'] = pd.to_datetime(df['When'])
df.tail()

,Year,Mo,Day,HH,MM,Tair,RH,Wind,Rad,Pressure,Prec,When,DoY
10964,2018,1,9,5,0,0.4,96.6,1.5,0.0,826.1,3.2,2018-01-09 05:00:00,9
10965,2018,1,9,6,0,0.3,97.2,0.7,0.0,825.8,0.7,2018-01-09 06:00:00,9
10966,2018,1,9,7,0,0.3,92.9,2.9,0.0,825.9,1.0,2018-01-09 07:00:00,9
10967,2018,1,9,8,0,-0.2,94.1,1.0,30.0,826.1,0.0,2018-01-09 08:00:00,9
10968,2018,1,9,9,20,-0.2,94.1,1.0,30.0,826.1,0.0,2018-01-09 09:00:00,9


In [5]:
# Cloudiness
clmax=np.zeros(shape=(366,24),dtype=float)

for i in range(0,365):
    for j in range(0,24):
        l=np.where(np.logical_and(np.logical_and(df['HH']==j, i-5<=df['DoY']), df['DoY']<=i+5))
        a=l[0]
        if a.size:
            clmax[i,j]=df.loc[a,'Rad'].max()/0.98

cl=np.empty(shape=df.shape[0],dtype=float)
for row in df.iterrows():
    a=clmax[df.loc[row[0],'DoY']-1,df.loc[row[0],'HH']]
    if a!=0:
        cl[row[0]]=(1-df.loc[row[0],'Rad']/a)
    # cloudiness at nighttime - use average of day before
    if df.loc[row[0],'Rad']<10:
        l=np.where(np.logical_and(np.logical_and(df['DoY']==df.loc[row[0],'DoY'], 10<=df['HH']), df['HH']<=15))
        b=np.array([])
        b=np.append(0,l)
        b=np.delete(b,0)
        if b.size:
            cl[row[0]]=np.mean(cl[b])
        else:
            cl[row[0]]=cl[row[0]-1]
df['Cl']=cl


In [6]:
#Select Data
mask = (df['When'] >= start_date) & (df['When'] < start_date+ timedelta(days=num_days))
df=df.loc[mask]
df=df.reset_index()

df['Tair']+=(elev_lake-hstat)*dtdz
df['Pressure']*=100
df['Prec']/=1000
df['rho_air']=df['Pressure']/(287.058*(df['Tair']+273.15))
df['ew']=6.1094*np.exp((17.625 * df['Tair'])/(df['Tair']+243.04))*(df['RH']/100)*100
df['huma']=0.622*df['ew']/abs(df['Pressure']-df['ew'])   # specific air humidity
df = df[['When','Rad', 'Tair', 'RH', 'Wind','Pressure','Prec','Cl','ew','rho_air','huma','DoY']]
df=df.round(5)
df.tail()

,When,Rad,Tair,RH,Wind,Pressure,Prec,Cl,ew,rho_air,huma,DoY
811,2018-01-08 19:00:00,0.0,-1.254,85.8,1.2,83020.0,0.0,0.56488,478.39521,1.06368,0.00360,8
812,2018-01-08 20:00:00,0.0,2.646,62.8,3.8,83020.0,0.0,0.56488,463.86898,1.04864,0.00349,8
813,2018-01-08 21:00:00,0.0,4.846,49.1,2.6,82990.0,0.0,0.56488,423.36901,1.03996,0.00319,8
814,2018-01-08 22:00:00,0.0,2.646,61.0,2.8,82960.0,0.0,0.56488,450.57338,1.04788,0.00340,8
815,2018-01-08 23:00:00,0.0,0.946,73.6,2.4,82930.0,0.0,0.56488,481.45391,1.05400,0.00363,8


In [7]:
#output csv
df.to_csv('../data/interim/weather.csv')